In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import pandas as pd
from model import CustomMultiHeadCNN
from utils import preprocess_image

# ----- Test Dataset -----
class TestDataset(Dataset):
    def __init__(self, images):
        self.images = torch.tensor(images, dtype=torch.float32).permute(0, 3, 1, 2)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx]





folder_path = "lots4"
all_items = os.listdir(folder_path)

filenames = sorted([item for item in all_items if os.path.isfile(os.path.join(folder_path, item))])
images_64 = []

for filename in filenames:
    path = os.path.join(folder_path, filename)
    img = preprocess_image(path, size=64)
    images_64.append(img)

images_64 = np.array(images_64)
print("Images shape:", images_64.shape)


# ----- Load model -----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CustomMultiHeadCNN(n_color=5, n_length=3).to(device)

checkpoint = torch.load("best_model_stabilized.pth", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()


# ----- Predictions -----
test_dataset = TestDataset(images_64)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

all_predictions = {'beard': [], 'mustache': [], 'glasses': [], 'hair_color': [], 'hair_length': []}

with torch.no_grad():
    for imgs in tqdm(test_loader, desc="Évaluation lot 3"):
        imgs = imgs.to(device)
        outputs = model(imgs)

        all_predictions['beard'].extend((torch.sigmoid(outputs['beard']) > 0.5).cpu().numpy().astype(int))
        all_predictions['mustache'].extend((torch.sigmoid(outputs['mustache']) > 0.5).cpu().numpy().astype(int))
        all_predictions['glasses'].extend((torch.sigmoid(outputs['glasses']) > 0.5).cpu().numpy().astype(int))
        all_predictions['hair_color'].extend(outputs['hair_color'].argmax(dim=1).cpu().numpy())
        all_predictions['hair_length'].extend(outputs['hair_length'].argmax(dim=1).cpu().numpy())

# ----- Save CSV -----
df = pd.DataFrame({
    "filename": [os.path.splitext(f)[0] for f in filenames],
    "barbe": all_predictions['beard'],
    "moustache": all_predictions['mustache'],
    "lunettes": all_predictions['glasses'],

"taille_cheveux": all_predictions['hair_length'],
    "couleur_cheveux": all_predictions['hair_color'],
      
})

df.to_csv("G10_L2.csv", index=False)
print("CSV généré : G10_L4.csv")
